# accompy Usage Demo

This notebook demonstrates the main features of accompy, following the README.

**accompy** generates backing track audio from chord charts in Python — like iReal Pro, but scriptable.

### First-time Setup

If you're using accompy for the first time, you can use the automated setup:

In [6]:
from accompy import verify_and_setup
verify_and_setup(interactive=True)

{'mma': False,
 'fluidsynth': True,
 'soundfont': True,
 'midiutil': True,
 'mingus': False}

This will:
- Check all dependencies
- Offer to install missing ones (with your permission)
- Download and configure SoundFont files
- Verify everything works

Let's check if everything is set up correctly

In [2]:
import os
os.environ["ACCOMPY_SKIP_SETUP_CHECK"] = "1"  # Skip warning for this demo

from accompy import print_diagnostic_report

print("Running diagnostic check...")
print_diagnostic_report()
# This will show detailed information about what's missing and how to fix it.

Running diagnostic check...
accompy Diagnostic Report

System: Darwin 24.6.0
Python: 3.12.12 (/Users/thorwhalen/.pyenv/versions/music_p12/bin/python)

--- Dependency Status ---
✗ mma: MISSING
✓ fluidsynth: available
✓ soundfont: available
✓ midiutil: available
✗ mingus: MISSING

--- Issues Found ---

1. mingus not installed (optional)
   Description: mingus provides better chord parsing and music theory support
   Solution: /Users/thorwhalen/.pyenv/versions/music_p12/bin/python -m pip install mingus



---
## 1. Basic Usage

The simplest way to generate a backing track is with a chord progression string.

In [3]:
from accompy import generate_accompaniment

# Generate with default settings
audio = generate_accompaniment("| C | Am | F | G |")
print(f"Generated: {audio}")

Generated: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmp235b0ux3.wav


### Specify Style and Tempo

You can customize the style, tempo, repeats, and output path:

In [ ]:
# Generate a swing backing track
audio = generate_accompaniment(
    "| Dm7 | G7 | Cmaj7 | Am7 |",
    style="swing",
    tempo=160,
    repeats=4,
    output_path="/tmp/swing_track.wav",
    backend="auto",  # auto|mma|builtin (auto prefers MMA if available)
 )

print(f"Generated swing track: {audio}")

Generated swing track: /tmp/swing_track.wav


## ensure_score

The tool that tries to convert whatever you give it to a "score" it can understand.

In [4]:
import os
os.environ["ACCOMPY_SKIP_SETUP_CHECK"] = "1"  # Skip warning for this demo

from accompy import ensure_score

# 1) A list of (chord, beats) tuples (like `accompany`)
chords = [("F#m7b5", 4), ("B7", 4), ("Em", 8)]
score_from_tuples = ensure_score(chords, key="E")
print("Score from (chord, beats):", score_from_tuples.measures)

# 2) iReal URLs can be passed directly (best results if pyRealParser is installed)
# url = "irealbook://Autumn%20Leaves=..."
# score_from_url = ensure_score(url)

# 3) MIDI-only output: only runs if `midiutil` is installed
from accompy import generate_accompaniment
try:
    midi_path = generate_accompaniment(chords, output_path="/tmp/accompy_demo.mid", backend="builtin")
    print("Generated MIDI:", midi_path)
except Exception as e:
    print("MIDI generation skipped:", e)

Score from (chord, beats): [['F#h7'], ['B7'], ['E-'], ['E-']]
Generated MIDI: /tmp/accompy_demo.mid


---
## 2. Using Score Objects

For more control, create a `Score` object with metadata and specific settings.

In [1]:
from accompy import Score, generate_accompaniment

# Parse chord string with metadata
score = Score.from_string(
    "| Dm7 | G7 | Cmaj7 | % |",  # % = repeat previous chord
    title="ii-V-I in C",
    key="C",
    time_signature=(4, 4)
)

# Generate audio from Score
audio = generate_accompaniment(score, style="swing", tempo=120)
print(f"Generated: {audio}")
print(f"Title: {score.title}")
print(f"Key: {score.key}")
print(f"Measures: {score.measures}")

Generated: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpk_9880v0.wav
Title: ii-V-I in C
Key: C
Measures: [['D-7'], ['G7'], ['C^7'], ['C^7']]


---
## 3. Chord Notation

accompy supports flexible chord notation including jazz shorthand (iReal Pro style).

### Jazz Notation (iReal Pro style)

In [2]:
# ^7=maj7, -7=min7
jazz_chords = "| C^7 | D-7 | G7 | C^7 |"
audio = generate_accompaniment(jazz_chords, style="swing", tempo=140)
print(f"Jazz notation: {audio}")

Jazz notation: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmp222knq0w.wav


### Standard Notation

In [11]:
standard_chords = "| Cmaj7 | Dm7 | G7 | Cmaj7 |"
audio = generate_accompaniment(standard_chords, style="bossa", tempo=120)
print(f"Standard notation: {audio}")

Standard notation: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmp_y5xuose.wav


### Multiple Chords Per Bar

In [12]:
# Two chords per bar, split evenly
multiple_chords = "| Dm7 G7 | Cmaj7 |"
audio = generate_accompaniment(multiple_chords, style="swing", tempo=160)
print(f"Multiple chords per bar: {audio}")

Multiple chords per bar: /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmpfmi_9qrv.wav


### Repeat Symbols

In [ ]:
# % repeats the previous chord
with_repeats = "| C | G | % | F |"
audio = generate_accompaniment(with_repeats, style="rock", tempo=120)
print(f"With repeats: {audio}")

### Chord Symbol Reference

| Notation | Meaning |
|----------|----------|
| `C`, `Cmaj` | C major |
| `C-`, `Cm`, `Cmin` | C minor |
| `C7` | C dominant 7 |
| `C^7`, `Cmaj7` | C major 7 |
| `C-7`, `Cm7`, `Cmin7` | C minor 7 |
| `Co`, `Cdim` | C diminished |
| `Co7`, `Cdim7` | C diminished 7 |
| `Ch7`, `Cm7b5` | C half-diminished |
| `C+`, `Caug` | C augmented |
| `Csus`, `Csus4` | C suspended 4th |

---
## 4. Musical Styles

accompy includes 8 different musical styles with characteristic patterns.

### Swing (Jazz)

Jazz swing with walking bass. Typical tempo: 120-200 BPM

In [14]:
swing = generate_accompaniment(
    "| Dm7 | G7 | Cmaj7 | A7 |",
    style="swing",
    tempo=160,
    output_path="/tmp/swing.wav"
)
print(f"Swing: {swing}")

Swing: /tmp/swing.wav


### Bossa Nova

Bossa nova with syncopated feel. Typical tempo: 100-150 BPM

In [15]:
bossa = generate_accompaniment(
    "| Dm7 | G7 | C^7 | A7b9 |",
    style="bossa",
    tempo=140,
    output_path="/tmp/bossa.wav"
)
print(f"Bossa: {bossa}")

Bossa: /tmp/bossa.wav


### Rock

Straight 8ths rock beat. Typical tempo: 100-140 BPM

In [ ]:
rock = generate_accompaniment(
    "| C | Am | F | G |",
    style="rock",
    tempo=120,
    output_path="/tmp/rock.wav"
)
print(f"Rock: {rock}")

### Funk

Syncopated funk groove. Typical tempo: 90-120 BPM

In [16]:
funk = generate_accompaniment(
    "| Em7 | A7 | Em7 | A7 |",
    style="funk",
    tempo=100,
    output_path="/tmp/funk.wav"
)
print(f"Funk: {funk}")

Funk: /tmp/funk.wav


### Ballad

Slow, sustained ballad. Typical tempo: 60-90 BPM

In [ ]:
ballad = generate_accompaniment(
    "| Cmaj7 | Am7 | Dm7 | G7 |",
    style="ballad",
    tempo=70,
    output_path="/tmp/ballad.wav"
)
print(f"Ballad: {ballad}")

### Latin

Latin/salsa with clave. Typical tempo: 100-130 BPM

In [22]:
latin = generate_accompaniment(
    "| Am7 | Dm7 | G7 | Cmaj7 |",
    style="latin",
    tempo=120,
    output_path="/tmp/latin.wav"
)
print(f"Latin: {latin}")

Latin: /tmp/latin.wav


### Waltz

3/4 waltz pattern. Typical tempo: 90-150 BPM

In [ ]:
# Note: You'd need to set time_signature=(3, 4) in the Score
waltz_score = Score.from_string(
    "| C | G | Am | F |",
    time_signature=(3, 4)
)
waltz = generate_accompaniment(
    waltz_score,
    style="waltz",
    tempo=120,
    output_path="/tmp/waltz.wav"
)
print(f"Waltz: {waltz}")

### Blues

Blues shuffle. Typical tempo: 80-120 BPM

In [17]:
# Classic 12-bar blues in A
blues = generate_accompaniment(
    "| A7 | A7 | A7 | A7 | D7 | D7 | A7 | A7 | E7 | D7 | A7 | E7 |",
    style="blues",
    tempo=100,
    output_path="/tmp/blues.wav"
)
print(f"Blues: {blues}")

Blues: /tmp/blues.wav


---
## 5. Advanced Configuration

Use `AccompanimentConfig` for full control over instruments, volumes, and output format.

In [ ]:
from accompy import AccompanimentConfig, generate_accompaniment

config = AccompanimentConfig(
    style="swing",
    tempo=160,
    repeats=4,
    instruments={
        "drums": True,
        "bass": False,
        "piano": False,
        "guitar": True,  # Disable guitar
    },
    volumes={
        "drums": 0.7,
        "bass": 1.0,
        "piano": 0.6,
    },
    sample_rate=44100,
    output_format="wav",  # Options: wav, mp3, flac, midi
 )

audio = generate_accompaniment(
    "| Dm7 | G7 | Cmaj7 | A7 |",
    config=config,
    output_path="/tmp/custom_track.wav",
 )

print(f"Generated with custom config: {audio}")

Generated with custom config: /tmp/custom_track.wav


### Custom SoundFont

You can specify a custom SoundFont file for different instrument sounds:

In [ ]:
config_with_custom_sf = AccompanimentConfig(
    soundfont=Path("/path/to/your/custom.sf2"),  # Custom SoundFont
    style="swing",
    tempo=120,
 )

# This would use your custom SoundFont
# audio = generate_accompaniment("| C | Am | F | G |", config=config_with_custom_sf)

---
## 6. Practical Examples

### Example 1: Jazz Standard (Autumn Leaves)

In [ ]:
autumn_leaves = Score.from_string(
    """| Cm7 | F7 | Bbmaj7 | Ebmaj7 | 
       | Am7b5 | D7 | Gm | Gm |""",
    title="Autumn Leaves (excerpt)",
    key="Bb"
)

audio = generate_accompaniment(
    autumn_leaves,
    style="swing",
    tempo=140,
    repeats=3,
    output_path="/tmp/autumn_leaves.wav"
)

print(f"Autumn Leaves: {audio}")

### Example 2: Pop Progression (I-V-vi-IV)

In [ ]:
pop_progression = "| C | G | Am | F |"

audio = generate_accompaniment(
    pop_progression,
    style="rock",
    tempo=120,
    repeats=4,
    output_path="/tmp/pop_progression.wav"
)

print(f"Pop progression: {audio}")

### Example 3: Bossa Nova (Girl from Ipanema style)

In [ ]:
bossa_changes = Score.from_string(
    """| Fmaj7 | Fmaj7 | G7 | G7 |
       | Gm7 | Gb7 | Fmaj7 | Gb7 |""",
    title="Bossa Nova Changes",
    key="F"
)

audio = generate_accompaniment(
    bossa_changes,
    style="bossa",
    tempo=130,
    output_path="/tmp/bossa_changes.wav"
)

print(f"Bossa changes: {audio}")

---
## 7. Diagnostic and Troubleshooting

If you encounter any issues, use these diagnostic tools.

### Full Diagnostic Report

In [ ]:
from accompy import print_diagnostic_report

print_diagnostic_report()

### Check Specific Issues

In [ ]:
from accompy import diagnose_issues

issues = diagnose_issues()

if issues:
    print("Issues found:")
    for issue, description, solution in issues:
        print(f"\n{issue}:")
        print(f"  Description: {description}")
        print(f"  Solution: {solution}")
else:
    print("✓ No issues found - everything is configured correctly!")

### Verify Individual Dependencies

In [ ]:
from accompy import check_dependencies

deps = check_dependencies()

print(f"FluidSynth: {deps['fluidsynth']}")
print(f"SoundFont: {deps['soundfont']}")
print(f"midiutil: {deps['midiutil']}")
print(f"mingus: {deps['mingus']}")
print(f"midi2audio: {deps['midi2audio']}")
print(f"MMA: {deps['mma']}")

---
## Summary

You've learned:

1. **Basic usage** - Simple chord strings to audio
2. **Score objects** - More control with metadata
3. **Chord notation** - Jazz, standard, and shorthand notation
4. **Musical styles** - 8 different styles from swing to blues
5. **Advanced config** - Custom instruments, volumes, and formats
6. **Practical examples** - Real-world chord progressions
7. **Diagnostics** - Troubleshooting and setup verification

**Next steps:**
- Try different chord progressions from your favorite songs
- Experiment with different styles and tempos
- Create practice tracks for your instrument
- Check the README for more advanced features